<a href="https://colab.research.google.com/github/TajbeerAhamed/Data-Mining-Lab-Assignments/blob/main/Assignment_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ID : 2125051007

Name: Md Tajbeer Ahamed Rimon

In [ ]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cosine

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
movies = pd.read_csv("/content/drive/MyDrive/Data_Mining_LAb/Assignments/movies.csv")
ratings = pd.read_csv("/content/drive/MyDrive/Data_Mining_LAb/Assignments/ratings.csv")

In [ ]:
# Create a pivot table to represent movie ratings by users
movie_ratings = ratings.pivot_table(index='userId', columns='movieId', values='rating').fillna(0)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Calculate cosine similarity between movies
movie_similarity = cosine_similarity(movie_ratings.T)  # Transpose to compare movies

# Create a DataFrame for easier access
movie_similarity_df = pd.DataFrame(movie_similarity, index=movie_ratings.columns, columns=movie_ratings.columns)

In [ ]:
def get_movie_recommendations(movie_id, top_n=10):
    """Returns the top N similar movies to a given movie."""
    similar_movies = movie_similarity_df[movie_id].sort_values(ascending=False)[1:]  # Exclude the movie itself
    top_similar_movies = similar_movies.head(top_n)

    # Get movie titles
    recommendations = movies[movies['movieId'].isin(top_similar_movies.index)]['title'].tolist()
    return recommendations

# Example usage:
movie_id_to_recommend = 1721  # Replace with the movie ID you want recommendations for
recommendations = get_movie_recommendations(movie_id_to_recommend)
print(f"Recommendations for movie ID {movie_id_to_recommend}: {recommendations}")

Recommendations for movie ID 1721: ['Men in Black (a.k.a. MIB) (1997)', 'Truman Show, The (1998)', 'Good Will Hunting (1997)', 'Saving Private Ryan (1998)', 'Star Wars: Episode I - The Phantom Menace (1999)', 'Sixth Sense, The (1999)', 'Shrek (2001)', 'Catch Me If You Can (2002)', 'Finding Nemo (2003)', 'Pirates of the Caribbean: The Curse of the Black Pearl (2003)']


In [ ]:
def get_personalized_recommendations(user_id, top_n=20):
    """Returns personalized recommendations for a user."""
    user_ratings = movie_ratings.loc[user_id]

    # Movies the user has rated highly (e.g., rating >= 4)
    liked_movies = user_ratings[user_ratings >= 4].index

    # Movies the user hasn't rated yet
    unrated_movies = user_ratings[user_ratings == 0].index

    recommendations = []
    for movie_id in liked_movies:
        similar_movies = movie_similarity_df[movie_id].sort_values(ascending=False)[1:] # Exclude the movie itself
        # Consider only unrated movies
        similar_unrated_movies = similar_movies[similar_movies.index.isin(unrated_movies)]
        recommendations.extend(similar_unrated_movies.head(top_n).index.tolist())

    # Get unique recommendations and their average ratings
    unique_recommendations = list(set(recommendations))
    average_ratings = ratings[ratings['movieId'].isin(unique_recommendations)].groupby('movieId')['rating'].mean()

    # Sort by average rating
    sorted_recommendations = average_ratings.sort_values(ascending=False).head(top_n).index.tolist()

    # Get movie titles
    recommended_movie_titles = movies[movies['movieId'].isin(sorted_recommendations)]['title'].tolist()
    return recommended_movie_titles
# Example usage:
user_id_to_recommend = 50  # Replace with the user ID
personalized_recommendations = get_personalized_recommendations(user_id_to_recommend)
print(f"Personalized recommendations for user ID {user_id_to_recommend}: {personalized_recommendations}")


Personalized recommendations for user ID 50: ['My Man Godfrey (1957)', 'Unfaithfully Yours (1948)', 'Ex Drummer (2007)', 'Che: Part Two (2008)', 'Zeitgeist: Moving Forward (2011)', 'George Carlin: Back in Town (1996)', 'George Carlin: You Are All Diseased (1999)', 'SORI: Voice from the Heart (2016)', 'Death Note: Desu nôto (2006–2007)', "Last Year's Snow Was Falling (1983)", 'Investigation Held by Kolobki (1986)', 'Karlson Returns (1970)', 'Vacations in Prostokvashino (1980)', 'Winter in Prostokvashino (1984)', 'Priklyucheniya Kapitana Vrungelya (1979)', 'Vovka in the Kingdom of Far Far Away (1965)', 'Cheburashka (1971)', 'Wow! A Talking Fish! (1983)', 'Travels of an Ant (1983)', 'Blue Planet II (2017)']
